#Librerias

In [9]:
import pandas as pd
%pip install ftfy
%pip install plot_confusion_matrix
%pip install wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os

import ftfy
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
import nltk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

You should consider upgrading via the '/Users/nataliaricaurte/Desktop/Etapa2 - API/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/nataliaricaurte/Desktop/Etapa2 - API/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Using cached wordcloud-1.9.3-cp39-cp39-macosx_10_9_x86_64.whl (173 kB)
You should consider upgrading via the '/Users/nataliaricaurte/Desktop/Etapa2 - API/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nataliaricaurte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nataliaricaurte/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nataliaricaurte/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Creación Pipeline

In [24]:
from procesamiento import tokenizer
import os

#Funciones individuales
#----------------------------------------------------------------
# Carga datos
def cargar_datos_excel(excel_path):
    datos = pd.read_excel(excel_path, engine='openpyxl')
    return datos

def cargar_datos_csv(csv_path):
    datos = pd.read_csv(csv_path, encoding='latin1') 
    return datos

# Cambiar a CVS
def guardar_datos_csv(datos, output_path):
    datos.to_csv(output_path, index=False, encoding='utf-8')

# Corregir codificación texto
def corregir_codificacion(datos, columna):
    datos[columna] = datos[columna].apply(ftfy.fix_text)
    return datos

# imprimir palabras más frecuentes por categoría
def print_most_frequent_words(pipeline, datos, columna_ods, categorias_ods, n=10):
    tfidf_vectorizer = pipeline.named_steps['tfidf']
    palabras_frecuentes_por_ods = {}
    
    for ods in categorias_ods:
        texts_for_ods = datos[datos[columna_ods] == ods]['Textos_espanol']
        X = tfidf_vectorizer.transform(texts_for_ods)
        words = tfidf_vectorizer.get_feature_names_out()
        sums = X.sum(axis=0).A1
        word_freq = [(word, sums[idx]) for idx, word in enumerate(words)]
        word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
        palabras_frecuentes_por_ods[ods] = word_freq[:n]  
    
    return palabras_frecuentes_por_ods
        
#----------------------------------------------------------------


#Evaluación de metricas del modelo de clasificación
def evaluar_modelo(y_test, y_pred):

    print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
    print("Recall: {}".format(recall_score(y_test, y_pred, average='macro')))
    print("Precisión: {}".format(precision_score(y_test, y_pred, average='macro')))
    print("Puntuación F1: {}".format(f1_score(y_test, y_pred, average='macro')))

    print("\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred))

    # Creación Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)


# Pipeline de procesamiento de datos
def pipeline_procesamiento_datos(data, columna_texto, columna_ods):
   
    if isinstance(data, str) and data.endswith('.csv'):
        datos = cargar_datos_csv(data)
    elif isinstance(data, str) and data.endswith('.xlsx'):
        datos = cargar_datos_excel(data)
    elif isinstance(data, pd.DataFrame):
        datos = data
    else:
        raise ValueError("El tipo de 'data' no es válido. Debe ser una ruta de archivo o un DataFrame.")

    # Creación del pipeline de preprocesamiento y modelo
    pipeline = Pipeline(steps=[
        ('tfidf', TfidfVectorizer(tokenizer=tokenizer)),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=0))
    ])

    # Dividir los datos en características (X) y etiquetas (y)
    X_data = datos[columna_texto]
    y_data = datos[columna_ods]

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

    # Ajustar el pipeline
    pipeline.fit(X_train, y_train)

    # Hacer predicciones en el conjunto de prueba
    y_pred = pipeline.predict(X_test)

    # Evaluar el modelo
    evaluar_modelo(y_test, y_pred)

    # Imprimir las palabras más frecuentes para cada categoría
    frecuentes = print_most_frequent_words(pipeline, datos, columna_ods='sdg', categorias_ods=[3, 4, 5], n=10)

    #print(frecuentes)
    # calcular metricas 
    metricas = {
        'accuracy': accuracy_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred, average='macro'),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'f1_score': f1_score(y_test, y_pred, average='macro')
    }

    print(frecuentes)

    joblib.dump(pipeline, 'pipeline.joblib')
    return {
        'metricas': metricas, 
        'frecuentes': frecuentes,
    }

excel_path = '/Users/nataliaricaurte/Desktop/Etapa2 - API/Back/ODScat_345.xlsx'
datos_corregidos = pipeline_procesamiento_datos(excel_path, 'Textos_espanol', 'sdg')

# Cargar el modelo guardado en otra sesión
def cargar_modelo():
    # Cargar el vectorizador y el clasificador
    current_dir = os.getcwd()
    pipeline = joblib.load(os.path.join(current_dir, 'pipeline.joblib'))
    return pipeline

# Cargar el pipeline para hacer predicciones
pipeline_cargado = cargar_modelo()


/Users/nataliaricaurte/Desktop/Etapa2 - API/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Exactitud: 0.97
Recall: 0.9694926736182102
Precisión: 0.969206607318536
Puntuación F1: 0.969343426453166

Reporte de Clasificación:
              precision    recall  f1-score   support

           3       0.97      0.98      0.97       250
           4       0.97      0.97      0.97       268
           5       0.97      0.96      0.97       292

    accuracy                           0.97       810
   macro avg       0.97      0.97      0.97       810
weighted avg       0.97      0.97      0.97       810

{3: [('', 307.93683581416553), ('salud', 82.85626706255475), ('atención', 65.98588620264049), ('servicios', 37.22729799610249), ('mental', 28.975696396482956), ('pacientes', 28.46306254028584), ('países', 24.560702519150418), ('primaria', 24.194617948142337), ('médicos', 24.014581478185423), ('enfermedades', 22.904032042662)], 4: [('', 333.44265082881844), ('educación', 66.28929006589918), ('estudiantes', 59.99506258491215), ('escuelas', 49.78338518397833), ('aprendizaje', 38.740287